In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
%matplotlib inline

In [ ]:
pip install pyforest

In [ ]:
from pyforest import *
lazy_imports()

In [ ]:
data = pd.read_csv('reverse.csv')

In [ ]:
# Prepare the data
X = data[['Compressive Strength (MPa)']]
y_wc = data['w/c']
y_fc = data['f/c']
y_cf = data['c/f']

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# Split the data for water-cement ratio prediction
X_train_wc, X_test_wc, y_train_wc, y_test_wc = train_test_split(X, y_wc, test_size=0.2, random_state=42)
# Split the data for fine-coarse aggregate ratio prediction
X_train_fc, X_test_fc, y_train_fc, y_test_fc = train_test_split(X, y_fc, test_size=0.2, random_state=42)
# Split the data for cement-fine aggregate ratio prediction
X_train_cf, X_test_cf, y_train_cf, y_test_cf = train_test_split(X, y_cf, test_size=0.2, random_state=42)

In [ ]:
# Initialize the Random Forest models
rf_wc = RandomForestRegressor(n_estimators=100, random_state=42)
rf_fc = RandomForestRegressor(n_estimators=100, random_state=42)
rf_cf = RandomForestRegressor(n_estimators=100, random_state=42)

In [ ]:
# Train the models
rf_wc.fit(X_train_wc, y_train_wc)
rf_fc.fit(X_train_fc, y_train_fc)
rf_cf.fit(X_train_cf, y_train_cf)

In [ ]:
# Make predictions and evaluate
wc_predictions = rf_wc.predict(X_test_wc)
fc_predictions = rf_fc.predict(X_test_fc)
cf_predictions = rf_cf.predict(X_test_cf)

In [ ]:
# Calculate and print MSE for evaluations
wc_mse = mean_squared_error(y_test_wc, wc_predictions)
fc_mse = mean_squared_error(y_test_fc, fc_predictions)
cf_mse = mean_squared_error(y_test_cf, cf_predictions)
wc_initial_rmse = np.sqrt(mean_squared_error(y_test_wc, wc_predictions))
fc_initial_rmse = np.sqrt(mean_squared_error(y_test_fc, fc_predictions))
cf_initial_rmse = np.sqrt(mean_squared_error(y_test_cf, cf_predictions))

In [ ]:
print(f'Water-Cement Ratio Prediction MSE: {wc_mse}')
print(f'Fine to Coarse Aggregate Ratio Prediction MSE: {fc_mse}')
print(f'Cement to Fine Aggregate Ratio Prediction MSE: {cf_mse}')
print(f'RMSE for water/cement = {wc_initial_rmse}')
print(f'RMSE for fine/coarse = {fc_initial_rmse}')
print(f'RMSE for cement/fine = {cf_initial_rmse}')

In [ ]:
# Function to predict ratios and calculate concrete components based on compressive strength
def predict_and_calculate_ratios(compressive_strength):
    input_data = pd.DataFrame({'Compressive Strength (MPa)': [compressive_strength]})
    predicted_wc = rf_wc.predict(input_data)[0]
    predicted_fc = rf_fc.predict(input_data)[0]
    predicted_cf = rf_cf.predict(input_data)[0]

    # Calculate the components based on the predicted ratios
    cement = round(1, 3)  # Assume cement as 1 unit
    water = round(predicted_wc * cement, 3) # water = w/c ratio * cement
    fine_aggregate = round(cement / predicted_cf, 3)  # fine_aggregate = cement / (c/f)
    coarse_aggregate = round(fine_aggregate / predicted_fc, 3) # coarse_aggregate = fine_aggregate / (f/c)

    return (water, cement, fine_aggregate, coarse_aggregate)

# Manual input of compressive strength
# compressive_strength = float(input("Enter the compressive strength in MPa: "))
# water, cement, fine_aggregate, coarse_aggregate = predict_and_calculate_ratios(compressive_strength)

# print(f"Calculated Water : Cement : Fine Aggregate : Coarse Aggregate =   {water:.3f},  {cement:.3f},  {fine_aggregate:.3f},  {coarse_aggregate:.3f}")

In [ ]:
#saving best model
import pickle
savePath = r'models\reverse.sav'
pickle.dump(rf_wc,rf_fc,rf_cf,open(savePath, 'wb'))